In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

# Input data files are available in the read-only "../input/" directory

import os
import numpy as np
import tensorflow as tf
import cv2

In [2]:
from tensorflow.keras.losses import binary_crossentropy

smooth = 1e-15

In [3]:
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + smooth) / (union + smooth)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)

def focal_loss(y_true, y_pred):
    alpha=0.25
    gamma=2
    def focal_loss_with_logits(logits, targets, alpha, gamma, y_pred):
        weight_a = alpha * (1 - y_pred) ** gamma * targets
        weight_b = (1 - alpha) * y_pred ** gamma * (1 - targets)
        return (tf.math.log1p(tf.exp(-tf.abs(logits))) + tf.nn.relu(-logits)) * (weight_a + weight_b) + logits * weight_b

    y_pred = tf.clip_by_value(y_pred, tf.keras.backend.epsilon(), 1 - tf.keras.backend.epsilon())
    logits = tf.math.log(y_pred / (1 - y_pred))
    loss = focal_loss_with_logits(logits=logits, targets=y_true, alpha=alpha, gamma=gamma, y_pred=y_pred)
    # or reduce_sum and/or axis=-1
    return tf.reduce_mean(loss)

In [4]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.applications import *

In [5]:
def squeeze_excite_block(inputs, ratio=8):
    init = inputs
    channel_axis = -1
    filters = init.shape[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)

    x = Multiply()([init, se])
    return x

def conv_block(inputs, filters):
    x = inputs

    x = Conv2D(filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = squeeze_excite_block(x)

    return x

def encoder1(inputs):
    skip_connections = []

    print(f'*************{inputs}*************')
    model = VGG19(include_top=False, weights='imagenet', input_tensor=inputs, input_shape=(288,384,3))
    names = ["block1_conv2", "block2_conv2", "block3_conv4", "block4_conv4"]
    for name in names:
        skip_connections.append(model.get_layer(name).output)

    output = model.get_layer("block5_conv4").output
    return output, skip_connections

def decoder1(inputs, skip_connections):
    num_filters = [256, 128, 64, 32]
    skip_connections.reverse()
    x = inputs

    for i, f in enumerate(num_filters):
        x = UpSampling2D((2, 2), interpolation='bilinear')(x)
        x = Concatenate()([x, skip_connections[i]])
        x = conv_block(x, f)

    return x

# def encoder2(inputs):
#     skip_connections = []
#
#     output = DenseNet121(include_top=False, weights='imagenet')(inputs)
#     model = tf.keras.models.Model(inputs, output)
#
#     names = ["input_2", "conv1/relu", "pool2_conv", "pool3_conv"]
#     for name in names:
#         skip_connections.append(model.get_layer(name).output)
#     output = model.get_layer("pool4_conv").output
#
#     return output, skip_connections

def encoder2(inputs):
    num_filters = [32, 64, 128, 256]
    skip_connections = []
    x = inputs

    for i, f in enumerate(num_filters):
        x = conv_block(x, f)
        skip_connections.append(x)
        x = MaxPool2D((2, 2))(x)

    return x, skip_connections

def decoder2(inputs, skip_1, skip_2):
    num_filters = [256, 128, 64, 32]
    skip_2.reverse()
    x = inputs

    for i, f in enumerate(num_filters):
        x = UpSampling2D((2, 2), interpolation='bilinear')(x)
        x = Concatenate()([x, skip_1[i], skip_2[i]])
        x = conv_block(x, f)

    return x

def output_block(inputs):
    x = Conv2D(1, (1, 1), padding="same")(inputs)
    x = Activation('sigmoid')(x)
    return x

def Upsample(tensor, size):
    """Bilinear upsampling"""
    def _upsample(x, size):
        return tf.image.resize(images=x, size=size)
    return Lambda(lambda x: _upsample(x, size), output_shape=size)(tensor)

def ASPP(x, filter):
    shape = x.shape

    y1 = AveragePooling2D(pool_size=(shape[1], shape[2]))(x)
    y1 = Conv2D(filter, 1, padding="same")(y1)
    y1 = BatchNormalization()(y1)
    y1 = Activation("relu")(y1)
    y1 = UpSampling2D((shape[1], shape[2]), interpolation='bilinear')(y1)

    y2 = Conv2D(filter, 1, dilation_rate=1, padding="same", use_bias=False)(x)
    y2 = BatchNormalization()(y2)
    y2 = Activation("relu")(y2)

    y3 = Conv2D(filter, 3, dilation_rate=6, padding="same", use_bias=False)(x)
    y3 = BatchNormalization()(y3)
    y3 = Activation("relu")(y3)

    y4 = Conv2D(filter, 3, dilation_rate=12, padding="same", use_bias=False)(x)
    y4 = BatchNormalization()(y4)
    y4 = Activation("relu")(y4)

    y5 = Conv2D(filter, 3, dilation_rate=18, padding="same", use_bias=False)(x)
    y5 = BatchNormalization()(y5)
    y5 = Activation("relu")(y5)

    y = Concatenate()([y1, y2, y3, y4, y5])

    y = Conv2D(filter, 1, dilation_rate=1, padding="same", use_bias=False)(y)
    y = BatchNormalization()(y)
    y = Activation("relu")(y)

    return y

# Function called in train.py
def build_model(shape):
    inputs = Input(shape)
    x, skip_1 = encoder1(inputs)
    x = ASPP(x, 64)
    x = decoder1(x, skip_1)
    outputs1 = output_block(x)

    x = inputs * outputs1

    x, skip_2 = encoder2(x)
    x = ASPP(x, 64)
    x = decoder2(x, skip_1, skip_2)
    outputs2 = output_block(x)
    outputs = Concatenate()([outputs1, outputs2])

    model = Model(inputs, outputs)
    return model

In [6]:
import json
from glob import glob
from sklearn.utils import shuffle
from tensorflow.keras.utils import CustomObjectScope
from tensorflow.keras.models import load_model

In [7]:
def create_dir(path):
    """ Create a directory. """
    try:
        if not os.path.exists(path):
            os.makedirs(path)
    except OSError:
        print(f"Error: creating directory with name {path}")

def read_data(impath, mskpath):
    """ Read the image and mask from the given path. """
    image = cv2.imread(impath, cv2.IMREAD_COLOR)
    mask = cv2.imread(mskpath, cv2.IMREAD_COLOR)
    return image, mask

def read_params():
    """ Reading the parameters from the JSON file."""
    with open("params.json", "r") as f:
        data = f.read()
        params = json.loads(data)
        return params

def load_data(path):
    """ Loading the data from the given path. """
    images_path = os.path.join(path, "image/*")
    masks_path  = os.path.join(path, "mask/*")

    images = glob(images_path)
    masks  = glob(masks_path)

    return images, masks

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

def load_model_weight(path):
    with CustomObjectScope({
        'dice_loss': dice_loss,
        'dice_coef': dice_coef,
        'bce_dice_loss': bce_dice_loss,
        'focal_loss': focal_loss,
        'iou': iou
        }):
        model = load_model(path)
    return model
    # model = build_model(256)
    # model.load_weights(path)
    # return model

In [8]:
from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers import Adam, Nadam
from tensorflow.keras.metrics import *
from sklearn.model_selection import train_test_split

In [9]:
def read_image(x):
    x = x.decode()
    image = cv2.imread(x, cv2.IMREAD_COLOR)
    image = np.clip(image - np.median(image)+127, 0, 255)
    image = image/255.0
    image = image.astype(np.float32)
    return image

def read_mask(y):
    y = y.decode()
    mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
    mask = mask/255.0
    mask = mask.astype(np.float32)
    mask = np.expand_dims(mask, axis=-1)
    return mask

def parse_data(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        y = np.concatenate([y, y], axis=-1)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([288, 384, 3])
    y.set_shape([288, 384, 2])
    return x, y

def tf_dataset(x, y, batch=8, isTrain=True):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    if isTrain:
        dataset = dataset.shuffle(buffer_size=32)
        dataset = dataset.map(map_func=parse_data)
        dataset = dataset.repeat()
        dataset = dataset.batch(batch)
    else:
        dataset = dataset.map(map_func=parse_data)
        dataset = dataset.batch(batch)
    return dataset

In [10]:
from tqdm import tqdm
from albumentations import (
    PadIfNeeded,
    HorizontalFlip,
    VerticalFlip,
    CenterCrop,
    Crop,
    Compose,
    Transpose,
    RandomRotate90,
    ElasticTransform,
    GridDistortion,
    OpticalDistortion,
    RandomSizedCrop,
    OneOf,
    CLAHE,
    RandomBrightnessContrast,
    RandomGamma,
    HueSaturationValue,
    RGBShift,
    RandomBrightness,
    RandomContrast,
    MotionBlur,
    MedianBlur,
    GaussianBlur,
    GaussNoise,
    ChannelShuffle,
    CoarseDropout
)

In [11]:
def augment_data(images, masks, save_path, augment=True):
    """ Performing data augmentation. """
    crop_size = (288-32, 384-32)
    size = (384,288)

    for image, mask in tqdm(zip(images, masks), total=len(images)):
        try:
            # Extracting just the name of the image from its path,
            # even excluding the file extension
            image_name = image.split("/")[-1].split(".")[0]
            mask_name = mask.split("/")[-1].split(".")[0]

            # retrieving an image and a mask from the given paths 'image' and 'mask'
            x, y = read_data(image, mask)
            try:
                h, w, c = x.shape
            except Exception as e:
                image = image[:-1]
                x, y = read_data(image, mask)
                h, w, c = x.shape

            if augment == True:
                ## Center Crop
                aug = CenterCrop(p=1, height=crop_size[0], width=crop_size[1])
                augmented = aug(image=x, mask=y)
                x1 = augmented['image']
                y1 = augmented['mask']

                ## Crop
                x_min = 0
                y_min = 0
                x_max = x_min + size[0]
                y_max = y_min + size[1]

                aug = Crop(p=1, x_min=x_min, x_max=x_max, y_min=y_min, y_max=y_max)
                augmented = aug(image=x, mask=y)
                x2 = augmented['image']
                y2 = augmented['mask']

                ## Random Rotate 90 degree
                aug = RandomRotate90(p=1)
                augmented = aug(image=x, mask=y)
                x3 = augmented['image']
                y3 = augmented['mask']

                ## Transpose
                aug = Transpose(p=1)
                augmented = aug(image=x, mask=y)
                x4 = augmented['image']
                y4 = augmented['mask']

                ## ElasticTransform
                aug = ElasticTransform(p=1, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03)
                augmented = aug(image=x, mask=y)
                x5 = augmented['image']
                y5 = augmented['mask']

                ## Grid Distortion
                aug = GridDistortion(p=1)
                augmented = aug(image=x, mask=y)
                x6 = augmented['image']
                y6 = augmented['mask']

                ## Optical Distortion
                aug = OpticalDistortion(p=1, distort_limit=2, shift_limit=0.5)
                augmented = aug(image=x, mask=y)
                x7 = augmented['image']
                y7 = augmented['mask']

                ## Vertical Flip
                aug = VerticalFlip(p=1)
                augmented = aug(image=x, mask=y)
                x8 = augmented['image']
                y8 = augmented['mask']

                ## Horizontal Flip
                aug = HorizontalFlip(p=1)
                augmented = aug(image=x, mask=y)
                x9 = augmented['image']
                y9 = augmented['mask']

                ## Grayscale
                x10 = cv2.cvtColor(x, cv2.COLOR_RGB2GRAY)
                y10 = y

                ## Grayscale Vertical Flip
                aug = VerticalFlip(p=1)
                augmented = aug(image=x10, mask=y10)
                x11 = augmented['image']
                y11 = augmented['mask']

                ## Grayscale Horizontal Flip
                aug = HorizontalFlip(p=1)
                augmented = aug(image=x10, mask=y10)
                x12 = augmented['image']
                y12 = augmented['mask']

                ## Grayscale Center Crop
                aug = CenterCrop(p=1, height=crop_size[0], width=crop_size[1])
                augmented = aug(image=x10, mask=y10)
                x13 = augmented['image']
                y13 = augmented['mask']

                ##
                aug = RandomBrightnessContrast(p=1)
                augmented = aug(image=x, mask=y)
                x14 = augmented['image']
                y14 = augmented['mask']

                aug = RandomGamma(p=1)
                augmented = aug(image=x, mask=y)
                x15 = augmented['image']
                y15 = augmented['mask']

                aug = HueSaturationValue(p=1)
                augmented = aug(image=x, mask=y)
                x16 = augmented['image']
                y16 = augmented['mask']

                aug = RGBShift(p=1)
                augmented = aug(image=x, mask=y)
                x17 = augmented['image']
                y17 = augmented['mask']

                aug = RandomBrightness(p=1)
                augmented = aug(image=x, mask=y)
                x18 = augmented['image']
                y18 = augmented['mask']

                aug = RandomContrast(p=1)
                augmented = aug(image=x, mask=y)
                x19 = augmented['image']
                y19 = augmented['mask']

                aug = MotionBlur(p=1, blur_limit=7)
                augmented = aug(image=x, mask=y)
                x20 = augmented['image']
                y20 = augmented['mask']

                aug = MedianBlur(p=1, blur_limit=9)
                augmented = aug(image=x, mask=y)
                x21 = augmented['image']
                y21 = augmented['mask']

                aug = GaussianBlur(p=1, blur_limit=9)
                augmented = aug(image=x, mask=y)
                x22 = augmented['image']
                y22 = augmented['mask']

                aug = GaussNoise(p=1)
                augmented = aug(image=x, mask=y)
                x23 = augmented['image']
                y23 = augmented['mask']

                aug = ChannelShuffle(p=1)
                augmented = aug(image=x, mask=y)
                x24 = augmented['image']
                y24 = augmented['mask']

                aug = CoarseDropout(p=1, max_holes=8, max_height=32, max_width=32)
                augmented = aug(image=x, mask=y)
                x25 = augmented['image']
                y25 = augmented['mask']

                images = [
                    x, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10,
                    x11, x12, x13, x14, x15, x16, x17, x18, x19, x20,
                    x21, x22, x23, x24, x25
                ]
                masks  = [
                    y, y1, y2, y3, y4, y5, y6, y7, y8, y9, y10,
                    y11, y12, y13, y14, y15, y16, y17, y18, y19, y20,
                    y21, y22, y23, y24, y25
                ]

            else:
                images = [x]
                masks  = [y]

            idx = 0
            for i, m in zip(images, masks):
                i = cv2.resize(i, size)
                m = cv2.resize(m, size)

                tmp_image_name = f"{image_name}_{idx}.jpg"
                tmp_mask_name  = f"{mask_name}_{idx}.jpg"

                image_path = os.path.join(save_path, tmp_image_name)
                mask_path  = os.path.join(save_path, tmp_mask_name)

                cv2.imwrite(image_path, i)
                cv2.imwrite(mask_path, m)

                idx += 1
        except:
            print('Invalid',image_name,mask_name)

def load_data():
    """ Load all the data and then split them into train and valid dataset. """
    img_path = glob("./kvasir_segmentation_dataset/images/*")
    msk_path = glob("./kvasir_segmentation_dataset/masks/*")

    img_path.sort()
    msk_path.sort()

    len_ids = len(img_path)
    train_size = int((80/100)*len_ids)
    valid_size = int((10/100)*len_ids)		## Here 10 is the percent of images used for validation
    test_size = int((10/100)*len_ids)		## Here 10 is the percent of images used for testing

    train_x, test_x = train_test_split(img_path, test_size=test_size, random_state=42)
    train_y, test_y = train_test_split(msk_path, test_size=test_size, random_state=42)

    train_x, valid_x = train_test_split(train_x, test_size=valid_size, random_state=42)
    train_y, valid_y = train_test_split(train_y, test_size=valid_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

In [12]:
# np.random.seed(42)
# path = "./kvasir_segmentation_dataset/"
# (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data()

# create_dir("./new_data/train/images/")
# create_dir("./new_data/train/masks/")
# create_dir("./new_data/valid/images/")
# create_dir("./new_data/valid/masks/")
# create_dir("./new_data/test/images/")
# create_dir("./new_data/test/masks/")

# augment_data(train_x, train_y, "new_data/train/", augment=True)
# augment_data(valid_x, valid_y, "new_data/valid/", augment=False)
# augment_data(test_x, test_y, "new_data/test/", augment=False)

In [13]:
np.random.seed(42)
tf.random.set_seed(42)
create_dir("files")

train_path = "./new_data/train"
valid_path = "./new_data/valid"
test_path = "./new_data/test"

## Training
train_x = sorted(glob(os.path.join(train_path, "images", "*.jpg")))
train_y = sorted(glob(os.path.join(train_path, "masks", "*.jpg")))


# ## Shuffling
# train_x, train_y = shuffling(train_x, train_y)

## Validation
valid_x = sorted(glob(os.path.join(valid_path, "images", "*.jpg")))
valid_y = sorted(glob(os.path.join(valid_path, "masks", "*.jpg")))



## Testing
test_x = sorted(glob(os.path.join(test_path, "images", "*.jpg")))
test_y = sorted(glob(os.path.join(test_path, "masks", "*.jpg")))
print(test_y)



model_path = "files/model.h5"
batch_size = 4
# Total number of epochs that would have completed after this training session
epochs = 50
# Number of epochs which have been completed till now.
# Training will automatically start from the next epoch onwards.
# Most recently trained by Puru on 2/12/21 night uptil 60 epochs
init_epoch = 0
lr = 1e-5
shape = (288, 384, 3)

model = build_model(shape)



metrics = [
    dice_coef,
    iou,
    Recall(),
    Precision()
]

train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
print(train_dataset)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)
print(valid_dataset)
test_dataset = tf_dataset(test_x, test_y, batch=batch_size, isTrain=False)
print(test_dataset)
####################
#tf.compat.v1.RunOptions(report_tensor_allocations_upon_oom = True)
#tf.compat.v1.disable_eager_execution()
######################

model.compile(loss=dice_loss, optimizer=Nadam(lr), metrics=metrics)

callbacks = [
    ModelCheckpoint(model_path),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=20),
    CSVLogger("files/data.csv"),
    TensorBoard(),
    EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=False)
]

train_steps = (len(train_x)//batch_size)
valid_steps = (len(valid_x)//batch_size)

if len(train_x) % batch_size != 0:
    train_steps += 1

if len(valid_x) % batch_size != 0:
    valid_steps += 1
# model.fit(train_dataset,
#         epochs=epochs,
#         validation_data=valid_dataset,
#         steps_per_epoch=train_steps,
#         validation_steps=valid_steps,
#         callbacks=callbacks,
#         initial_epoch=init_epoch,
#         shuffle=False)

['./new_data/test\\masks\\cju0tl3uz8blh0993wxvn7ly3_0.jpg', './new_data/test\\masks\\cju15l5ubz9yh0855b3ivdpse_0.jpg', './new_data/test\\masks\\cju16d65tzw9d0799ouslsw25_0.jpg', './new_data/test\\masks\\cju17otoe119u0799nqcbl8n1_0.jpg', './new_data/test\\masks\\cju1ats0y372e08011yazcsxm_0.jpg', './new_data/test\\masks\\cju1bm8063nmh07996rsjjemq_0.jpg', './new_data/test\\masks\\cju1c4fcu40hl07992b8gj0c8_0.jpg', './new_data/test\\masks\\cju1cbokpuiw70988j4lq1fpi_0.jpg', './new_data/test\\masks\\cju1cdxvz48hw0801i0fjwcnk_0.jpg', './new_data/test\\masks\\cju1cnnziug1l0835yh4ropyg_0.jpg', './new_data/test\\masks\\cju1d31sp4d4k0878r3fr02ul_0.jpg', './new_data/test\\masks\\cju1efbr0rqxz09931z0lf4vf_0.jpg', './new_data/test\\masks\\cju1expq45zst0855rjqwwj4m_0.jpg', './new_data/test\\masks\\cju1f8w0t65en0799m9oacq0q_0.jpg', './new_data/test\\masks\\cju1fmsyf6gxb0801cimx2gle_0.jpg', './new_data/test\\masks\\cju2hugv9vget0799hhk7ksvg_0.jpg', './new_data/test\\masks\\cju2i03ptvkiu0799xbbd4det_0.jp

In [14]:
def read_image(x):
    image = cv2.imread(x, cv2.IMREAD_COLOR)
    image = np.clip(image - np.median(image)+127, 0, 255)
    image = image/255.0
    image = image.astype(np.float32)
    image = np.expand_dims(image, axis=0)
    return image

def read_mask(y):
    mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
    mask = mask.astype(np.float32)
    mask = mask/255.0
    mask = np.expand_dims(mask, axis=-1)
    return mask

def mask_to_3d(mask):
    mask = np.squeeze(mask)
    mask = [mask, mask, mask]
    mask = np.transpose(mask, (1, 2, 0))
    return mask

def parse(y_pred):
    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = y_pred[..., -1]
    y_pred = y_pred.astype(np.float32)
    y_pred = np.expand_dims(y_pred, axis=-1)
    return y_pred

def evaluate_normal(model, x_data, y_data):
    """stores results of predictions"""
    THRESHOLD = 0.5
    x = read_image(x_data)
    y = read_mask(y_data)
    _, h, w, _ = x.shape

    y_pred1 = parse(model.predict(x)[0][..., -2])
    y_pred2 = parse(model.predict(x)[0][..., -1])
    
    polarize = lambda x: 0 if x<0.5 else 1
    
    image1 = np.ndarray([288,384,1])
    image2 = np.ndarray([288,384,1])
    
    for row in range(y_pred1.shape[0]):        
        for column in range(y_pred1.shape[1]):
            image1[row][column][0] = polarize(y_pred1[row][column][0])
            
    for row in range(y_pred2.shape[0]):        
        for column in range(y_pred2.shape[1]):
            image2[row][column][0] = polarize(y_pred2[row][column][0])


    line = np.ones((h, 10, 3)) * 255.0

    all_images = [
        x[0] * 255.0, line,
        mask_to_3d(y) * 255.0, line,
        mask_to_3d(image1) * 255.0, line,
        mask_to_3d(image2) * 255.0
    ]
    
    mask = np.concatenate(all_images, axis=1)
    cv2.imwrite('./results/mask.jpg',mask)    

In [15]:
model.load_weights('./files/model.h5')
data_x = './new_data/test/images/cju0tl3uz8blh0993wxvn7ly3_0.jpg'
data_y = './new_data/test/masks/cju0tl3uz8blh0993wxvn7ly3_0.jpg'

# Writing the image result in the output directory
evaluate_normal(model, data_x, data_y)

1/1 [==============================] - 0s 38ms/step
